In [1]:
import os
import matplotlib.pyplot as plt
from pathlib import Path
import scanpy as sc
import pandas as pd

# from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
#                             homogeneity_completeness_v_measure
from sklearn.preprocessing import LabelEncoder
import time
import psutil
import tracemalloc
import sys
sys.path.append('/home/lytq/Spatial-Transcriptomics-Benchmark/DeepST/DeepST-main/deepst')
from DeepST import run

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/home/lytq/Spatial-Transcriptomics-Benchmark/utils')
from sdmbench import compute_ARI, compute_NMI, compute_CHAOS, compute_PAS, compute_ASW, compute_HOM, compute_COM


def evaluate_clustering(adata: sc.AnnData, df_meta, time_taken: float, memory_used: float, output_dir: str) -> dict:
    """Evaluate clustering using sdmbench"""
    gt_key = 'ground_truth'
    pred_key = 'DeepST_refine_domain'
    adata.obs['ground_truth'] = df_meta['fine_annot_type'].values
    adata = adata[~pd.isnull(adata.obs['ground_truth'])]
    
    results = {
        "ARI": compute_ARI(adata, gt_key, pred_key),
        "AMI": compute_NMI(adata, gt_key, pred_key),
        "Homogeneity": compute_HOM(adata, gt_key, pred_key),
        "Completeness": compute_COM(adata, gt_key, pred_key),
        "ASW": compute_ASW(adata, pred_key),
        "CHAOS": compute_CHAOS(adata, pred_key),
        "PAS": compute_PAS(adata, pred_key),
        "Time": time_taken,
        "Memory": memory_used
    }
    
    df_results = pd.DataFrame([results])
    df_results.to_csv(os.path.join(output_dir, "metrics.csv"), index=False)
    return results
    



In [6]:
data_path = "/home/lytq/Spatial-Transcriptomics-Benchmark/data/BRCA1" #### to your path
# sample = sys.argv[1]
data_name = 'V1_Human_Breast_Cancer_Block_A_Section_1'


save_root = Path(f'/home/lytq/Spatial-Transcriptomics-Benchmark/RESULTS/BRCA1/DeepST/')
os.makedirs(save_root, exist_ok=True)

start_time = time.time()
tracemalloc.start()

print('Processing', data_name, '...')
n_domains = 20
# n_domains = sys.argv[2]


Processing V1_Human_Breast_Cancer_Block_A_Section_1 ...


In [3]:
deepen = run(
    save_path = save_root,
    task = "Identify_Domain", #### DeepST includes two tasks, one is "Identify_Domain" and the other is "Integration"
    pre_epochs = 800, ####  choose the number of training
    epochs = 1000, #### choose the number of training
    use_gpu = True)


In [7]:
adata = deepen._get_adata(platform="Visium", data_path=data_path, data_name=data_name)


In [8]:
adata

AnnData object with n_obs × n_vars = 3798 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'imagecol', 'imagerow'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [13]:
adata.obs_names

Index(['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', 'AAACAGAGCGACTCCT-1',
       'AAACAGGGTCTATATT-1', 'AAACAGTGTTCCTGGG-1', 'AAACATTTCCCGGATT-1',
       'AAACCCGAACGAAATC-1', 'AAACCGGGTAGGTACC-1', 'AAACCTAAGCAGCCGG-1',
       'AAACCTCATGAAGTTG-1',
       ...
       'TTGTGGTAGGAGGGAT-1', 'TTGTGGTGGTACTAAG-1', 'TTGTGTATGCCACCAA-1',
       'TTGTTAGCAAATTCGA-1', 'TTGTTCAGTGTGCTAC-1', 'TTGTTGTGTGTCAAGA-1',
       'TTGTTTCACATCCAGG-1', 'TTGTTTCATTAGTCTA-1', 'TTGTTTCCATACAACT-1',
       'TTGTTTGTGTAAATTC-1'],
      dtype='object', length=3798)

In [14]:
gt_df = pd.read_csv(data_path + '/' + data_name + '/metadata.tsv', sep='\t', index_col=0)
gt_df

,annot_type,fine_annot_type
ID,,
AAACAAGTATCTCCCA-1,Surrounding tumor,Tumor_edge_5
AAACACCAATAACTGC-1,Invasive,IDC_4
AAACAGAGCGACTCCT-1,Healthy,Healthy_1
AAACAGGGTCTATATT-1,Invasive,IDC_3
AAACAGTGTTCCTGGG-1,Invasive,IDC_4
...,...,...
TTGTTGTGTGTCAAGA-1,Invasive,IDC_7
TTGTTTCACATCCAGG-1,Invasive,IDC_4
TTGTTTCATTAGTCTA-1,Invasive,IDC_4


In [15]:
gt_df.loc[adata.obs_names, 'fine_annot_type']


AAACAAGTATCTCCCA-1    Tumor_edge_5
AAACACCAATAACTGC-1           IDC_4
AAACAGAGCGACTCCT-1       Healthy_1
AAACAGGGTCTATATT-1           IDC_3
AAACAGTGTTCCTGGG-1           IDC_4
                          ...     
TTGTTGTGTGTCAAGA-1           IDC_7
TTGTTTCACATCCAGG-1           IDC_4
TTGTTTCATTAGTCTA-1           IDC_4
TTGTTTCCATACAACT-1    Tumor_edge_2
TTGTTTGTGTAAATTC-1    Tumor_edge_1
Name: fine_annot_type, Length: 3798, dtype: object

In [10]:
adata.obs['Ground Truth'] = gt_df.loc[adata.obs_names, 'fine_annot_type']

KeyError: "None of [Index(['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', 'AAACAGAGCGACTCCT-1',\n       'AAACAGGGTCTATATT-1', 'AAACAGTGTTCCTGGG-1', 'AAACATTTCCCGGATT-1',\n       'AAACCCGAACGAAATC-1', 'AAACCGGGTAGGTACC-1', 'AAACCTAAGCAGCCGG-1',\n       'AAACCTCATGAAGTTG-1',\n       ...\n       'TTGTGGTAGGAGGGAT-1', 'TTGTGGTGGTACTAAG-1', 'TTGTGTATGCCACCAA-1',\n       'TTGTTAGCAAATTCGA-1', 'TTGTTCAGTGTGCTAC-1', 'TTGTTGTGTGTCAAGA-1',\n       'TTGTTTCACATCCAGG-1', 'TTGTTTCATTAGTCTA-1', 'TTGTTTCCATACAACT-1',\n       'TTGTTTGTGTAAATTC-1'],\n      dtype='object', length=3798)] are in the [index]"

In [ ]:

adata.obs['Ground Truth'] = gt_df.loc[adata.obs_names, 'fine_annot_type']
adata.layers['count'] = adata.X.toarray()

###### Segment the Morphological Image
adata = deepen._get_image_crop(adata, data_name=data_name)

###### Data augmentation. spatial_type includes three kinds of "KDTree", "BallTree" and "LinearRegress", among which "LinearRegress"
###### is only applicable to 10x visium and the remaining omics selects the other two.
###### "use_morphological" defines whether to use morphological images.
adata = deepen._get_augment(adata, spatial_type="LinearRegress", use_morphological=True)

###### Build graphs. "distType" includes "KDTree", "BallTree", "kneighbors_graph", "Radius", etc., see adj.py
graph_dict = deepen._get_graph(adata.obsm["spatial"], distType = "BallTree")

###### Enhanced data preprocessing
data = deepen._data_process(adata, pca_n_comps = 200)

###### Training models
deepst_embed = deepen._fit(
    data = data,
    graph_dict = graph_dict,
)
# Remove Image_crop folder after training
os.system(f'rm -r {save_root}/Image_crop')

###### DeepST outputs
adata.obsm["DeepST_embed"] = deepst_embed

###### Define the number of space domains, and the model can also be customized. If it is a model custom priori = False.
adata = deepen._get_cluster_data(adata, n_domains=n_domains, priori = True)
# print(adata)


###### Calculate the time and memory used
time_taken = time.time() - start_time
size, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()
memory_used = peak / (1024 ** 2) # MB

####### Calculate clustering metrics
# obs_df = adata.obs.dropna()
clustering_results = evaluate_clustering(adata, gt_df, time_taken, memory_used, save_root)
clustering_results.to_csv(save_root / 'metrics.csv', index=False)

###### Spatial localization map of the spatial domain
fig, ax = plt.subplots(1, 1, figsize=(6, 6))        
sc.pl.spatial(adata, 
                color='DeepST_refine_domain',
                frameon = False, 
                spot_size=150, 
                title=f'DeepST (ARI = {clustering_results["ARI"]:.4f})',
                ax=ax)  
handles, labels = ax.get_legend_handles_labels()
new_labels = [str(int(label) + 1) if label.isdigit() else label for label in labels]
ax.legend(handles, new_labels, loc='center left', bbox_to_anchor=(1.05, 0.5), frameon=False) 
plt.savefig(os.path.join(save_root, f'clustering.pdf'), bbox_inches='tight', dpi=300)

adata.obs['DeepST'] = adata.obs['DeepST_refine_domain']

# adata.write(save_root / 'result.h5ad')
# df_PC = pd.DataFrame(data=adata.obsm['DeepST_embed'], index=adata.obs.index)
# df_PC.to_csv(save_root / 'PCs.tsv', sep='\t')
# adata.obs.to_csv( save_root / 'metadata.tsv', sep='\t')

###### UMAP visualization
sc.pp.neighbors(adata, use_rep='DeepST_embed', n_neighbors=10)
sc.tl.umap(adata)

fig, axes = plt.subplots(1,1,figsize=(4*2, 3))
sc.pl.umap(adata, color='Ground Truth', ax=axes[0], show=False)
sc.pl.umap(adata, color='DeepST_refine_domain', ax=axes[1], show=False)
axes[0].set_title('Manual Annotation')
axes[1].set_title('DeepST')
handles, labels = axes[1].get_legend_handles_labels()
new_labels = [str(int(label) + 1) if label.isdigit() else label for label in labels]
axes[1].legend(handles, new_labels, loc='center left', frameon=False, bbox_to_anchor=(1, 0.5))
for ax in axes:
    ax.set_aspect(1)
plt.tight_layout()
plt.savefig(os.path.join(save_root, f'umap.pdf'), bbox_inches='tight', dpi=300)

low_dim_data = pd.DataFrame(adata.obsm['image_feat'], index=adata.obs.index)
# expression_data = pd.DataFrame(adata.layers['count'], index=adata.obs.index, columns=adata.var.index)
cell_metadata = adata.obs

low_dim_data.to_csv(f"{save_root}/low_dim_data.csv", index=False)
# expression_data.T.to_csv(f"{save_root}/expression_matrix.csv")
cell_metadata.to_csv(f"{save_root}/cell_metadata.csv", index=False)

umap_coords = adata.obsm["X_umap"]
spot_ids = adata.obs_names
umap_df = pd.DataFrame(umap_coords, columns=["UMAP1", "UMAP2"])
umap_df["spot_id"] = spot_ids
umap_df = umap_df[["spot_id", "UMAP1", "UMAP2"]]
umap_df.to_csv(os.path.join(save_root, "spatial_umap_coords.csv"), index=False)

print('Done', data_name)
